# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,rio grande,-32.0350,-52.0986,23.06,84,100,2.19,BR,1711999219
1,1,waitangi,-43.9535,-176.5597,12.68,90,100,3.49,NZ,1711999219
2,2,castillos,-34.1667,-53.8333,14.37,89,100,5.00,UY,1711999220
3,3,longyearbyen,78.2186,15.6401,-16.09,72,75,1.54,SJ,1711999220
4,4,hawaiian paradise park,19.5933,-154.9731,25.49,79,20,1.54,US,1711999221


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points(
    'Lng',
    'Lat',
    geo = True,
    tiles = 'OSM',
    frame_width = 800,
    frame_height = 500,
    size='Humidity',
    color = "City"
)

# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values

# For my ideal destination, I will choose just cities with clear skies (cloudiness < 10),
# not too windy (wind speed < 7),
# and high temperatures between 20 and 26 C

# First, we will set the copy_on_write = 'True' value for pandas to eliminate an unnecessary warning
# when we drop our NA rows in a second
pd.options.mode.copy_on_write = True

# Now, on to setting up our slice of the city_data_df
city_data_df = city_data_df.loc[(city_data_df['Max Temp'] >= 20) & 
                                (city_data_df['Max Temp'] <= 26) & 
                                (city_data_df['Cloudiness'] <= 10) & 
                                (city_data_df['Wind Speed'] < 7)]

# Drop any rows with null values
city_data_df.dropna(axis=0, how='any', inplace=True)

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
26,26,geraldton,-28.7667,114.6000,24.23,41,5,5.66,AU,1711999180
79,79,phan rang-thap cham,11.5667,108.9833,24.73,86,8,1.23,VN,1711999254
130,130,coquimbo,-29.9533,-71.3436,20.85,60,0,5.66,CL,1711999023
134,134,lompoc,34.6391,-120.4579,21.50,63,0,4.12,US,1711999144
164,164,qui nhon,13.7667,109.2333,24.78,92,1,2.89,VN,1711999291


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = city_data_df.filter(['City', 'Country', 'Lat', 'Lng', 'Humidity'], axis=1).reset_index(drop=True)

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ''

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
0,geraldton,AU,-28.7667,114.6000,41,
1,phan rang-thap cham,VN,11.5667,108.9833,86,
2,coquimbo,CL,-29.9533,-71.3436,60,
3,lompoc,US,34.6391,-120.4579,63,
4,qui nhon,VN,13.7667,109.2333,92,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel

# I will want to filter by circle (using a radius)
# My bias can be by distance from the lat/long I submit
# so I will chose bias=circle:lon,lat,radius
#
# I want to look for hotels, so I will use categories for that. That means a category 
# of accommodation.hotel. I will want the hotel name and country as additional info
# so my final "hover over" should consist of lng, lat, city, humidity (whcih will still drive the dot size)
# hotel name, and hotel country

radius = 10000
categories = 'accommodation.hotel'
longitude = ''
latitude = ''
filters = ''
bias = f'proximity:{longitude},{latitude}'
limit = 1


params = {
    "categories":categories,
    "limit":limit,
    "filter":filters,
    "bias":bias,
    "apiKey":geoapify_key    
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    longitude = hotel_df.iloc[index,3]
    latitude = hotel_df.iloc[index,2]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params['filter'] = f'circle:{longitude},{latitude},{radius}'
    params['bias'] = f'proximity:{longitude},{latitude}'
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
geraldton - nearest hotel: Ocean Centre Hotel
phan rang-thap cham - nearest hotel: Khách Sạn Ninh Thuận
coquimbo - nearest hotel: Hotel Iberia
lompoc - nearest hotel: Red Roof Inn Lompoc
qui nhon - nearest hotel: TMS Quy Nhơn - Home ở đây!
port blair - nearest hotel: Andaman Ocean Breeze
tandil - nearest hotel: Viñas del Rosario Hotel Boutique
tura - nearest hotel: Hotel Polo Orchid
darnah - nearest hotel: فندق الفردوس
sur - nearest hotel: Sur Hotel
eureka - nearest hotel: Eureka Inn, Trademark Collection by Wyndham
yangshuo - nearest hotel: No hotel found
heyuan - nearest hotel: 汇景希尔顿逸林酒店
coronel dorrego - nearest hotel: Hotel América
bamboo flat - nearest hotel: Hotel Megapode Nest
kone - nearest hotel: Pacifik Appartels
qionghai - nearest hotel: 白玉蘭酒店（愛華路店）
el granada - nearest hotel: Beach House
laguna - nearest hotel: Holiday Inn Express & Suites
silistra - nearest hotel: Силистра
gaiman - nearest hotel: yr hen ffordd
pumiao - nearest hotel: 三丰酒店
carlsbad - n

,City,Country,Lat,Lng,Humidity,Hotel Name
0,geraldton,AU,-28.7667,114.6000,41,Ocean Centre Hotel
1,phan rang-thap cham,VN,11.5667,108.9833,86,Khách Sạn Ninh Thuận
2,coquimbo,CL,-29.9533,-71.3436,60,Hotel Iberia
3,lompoc,US,34.6391,-120.4579,63,Red Roof Inn Lompoc
4,qui nhon,VN,13.7667,109.2333,92,TMS Quy Nhơn - Home ở đây!
5,port blair,IN,11.6667,92.7500,94,Andaman Ocean Breeze
6,tandil,AR,-37.3217,-59.1332,33,Viñas del Rosario Hotel Boutique
7,tura,IN,25.5198,90.2201,34,Hotel Polo Orchid
8,darnah,LY,32.7670,22.6367,50,فندق الفردوس
9,sur,OM,22.5667,59.5289,56,Sur Hotel


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [8]:
%%capture --no-display

# Configure the map plot
map_plot = hotel_df.hvplot.points(
    'Lng',
    'Lat',
    geo = True,
    tiles = 'OSM',
    frame_width = 800,
    frame_height = 500,
    size='Humidity',
    hover_cols=['City', 'Hotel Name', 'Country'],
    color = "City"
)

# Display the map
map_plot


:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)